In [33]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install nltk tensorflow-hub

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [34]:
import preprocessor
import pandas as pd
import pickle
import os

In [35]:
def maybe_process(store_file="data.pkl", force=False):
    if force or not os.path.exists(store_file):
        data = preprocessor.process_dir("dump")
        with open(store_file, 'wb') as data_file:
            pickle.dump(data, data_file)
    else:
        with open(store_file, 'rb') as data_file:
            data = pickle.load(data_file)
    return data

In [36]:
data = maybe_process()

In [37]:
df = pd.DataFrame(data)
df = df.fillna("Unknown")

In [38]:
user_vocabulary = pd.concat([df["assignee"], df["reporter"]]).unique()

In [39]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)

In [40]:
vocabulary_map = {"Unknown": 0}
count = 0
for el in user_vocabulary:
    count += 1
    vocabulary_map[el] = count

In [41]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)

with open("vocabulary_map.pkl", 'wb') as vocabulary_file:
    pickle.dump(vocabulary_map, vocabulary_file)

Start tensorflow stuff here

In [42]:
import tensorflow as tf
import tensorflow_hub as hub
import shutil
import numpy as np

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

1.9.0


In [43]:
df.head()

,assignee,description,id,key,reporter,status,summary
0,yh02364,Broken links to account_xapi logs and conf on ...,825488,DEVOPS-8155,yh02364,Closed,Broken links to account_xapi logs and conf on ...
1,yh00677,https://c4d.devops.fds.com/#pk=8143 \r\n\r\nT...,683408,WDSDO-25052,p139rpd,Closed,Delete C4D Env
2,yc03k1m,Please grant access to Suvarna (m172665) tpo J...,753374,DEVOPS-2558,m910042,Closed,Suvarna Kommura requires QA access to Jenkins ...
3,yc14sv1,Parameters that used in deployment or redeploy...,299398,WDSDO-5800,yc14ay1,Closed,Request to provide parameters of Webapp enviro...
4,m825519,Create new certification dashboards for MCOM a...,638405,WDSDO-22797,m825519,Closed,Certification dashboard per brand


In [44]:
df.describe()

,assignee,description,id,key,reporter,status,summary
count,41397,41397,41397,41397,41397,41397,41397
unique,1121,40159,41397,41397,2151,15,39824
top,yc14tz1,TBD,781836,WDSDO-2919,yc14sv1,Closed,Discovery build failed
freq,3832,104,1,1,1002,36637,22


Now, split the data into two parts -- training and evaluation.

In [45]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

In [46]:
def add_more_features(df):
    # TODO: Add more features to the dataframe
    return df

In [47]:
# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = add_more_features(df),
#         y = df["assignee"].fillna('Unknown').apply(lambda x: vocabulary_map[x]),
        y = df["assignee"],
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000,
        num_threads = 1
      )

In [48]:
# Define your feature columns
def create_feature_cols():
    return [
#         tf.feature_column.categorical_column_with_vocabulary_list("reporter", 
#                                                                   vocabulary_list=user_vocabulary.tolist(), 
#                                                                   default_value=0)


        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                "reporter", vocabulary_list=user_vocabulary.tolist(), 
                default_value=0),
            9,
        ),
        
        hub.text_embedding_column(
            key="description", 
            module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
            trainable=True,
        ),
        
        hub.text_embedding_column(
            key="summary", 
            module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
            trainable=True,
        ),
        
    # TODO: Define additional feature columns
    # Hint: Are there any features that would benefit from bucketizing?
    ]

In [49]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNClassifier(
        model_dir = output_dir,
        feature_columns = create_feature_cols(),
        hidden_units=[500, 100],
        n_classes=len(user_vocabulary.tolist()),
        label_vocabulary = user_vocabulary.tolist(),
    )
    
#     estimator = tf.estimator.LinearClassifier(
#         model_dir = output_dir, 
#         feature_columns = create_feature_cols(),
#         n_classes=len(user_vocabulary.tolist()),
#         label_vocabulary = user_vocabulary.tolist(),
#     )
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, None),
                                        max_steps = num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(evaldf, 1), 
                                      steps = None, 
                                      start_delay_secs = 1, # start evaluating after N seconds, 
                                      throttle_secs = 30)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [50]:
OUTDIR = './trained_model'

In [ ]:
# Run the model
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 1000)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/nnlm-en-dim128/1'.
